In [3]:
with open('input.txt','r') as f:
    text = f.read()

In [4]:
len(text)

1115393

In [5]:
print(text[:1000])  # Print the first 1000 characters to verify content

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars =  sorted(list(set(text)))
vocab_size = len(chars)
print("All the unique characters:",''.join(chars))
print("Vocab size:",vocab_size)

All the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [7]:
# create a mapping from charaq
stoi = {ch :i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

In [10]:
stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [8]:
encode = lambda s : [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l : ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

In [9]:
print(encode("hello world"))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]


In [12]:
# encode the entire dataset 

import torch 
data = torch.tensor(encode(text),dtype=torch.long)

print(data.shape, data.dtype)

torch.Size([1115393]) torch.int64


In [13]:
n = int(0.9*len(data))  # first 90% will be train, rest validation
train_data = data[:n]   
val_data = data[n:]

In [14]:
block_size = 8 

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x,y = train_data[:block_size], train_data[1:block_size+1]

In [18]:
print(x,y)

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [16]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [19]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?


In [20]:
def get_batch(split):
    data = train_data if split == 'train' else val_data 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x  = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x,y

In [21]:
xb,yb = get_batch('train')
print('inputs:')
print(xb)
print(f"end={xb.shape}")
print('targets:')
print(yb)
print(f"end={yb.shape}")

inputs:
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
end=torch.Size([4, 8])
targets:
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
end=torch.Size([4, 8])


In [22]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is {target.item()}")

when input is [53] the target is 59
when input is [53, 59] the target is 6
when input is [53, 59, 6] the target is 1
when input is [53, 59, 6, 1] the target is 58
when input is [53, 59, 6, 1, 58] the target is 56
when input is [53, 59, 6, 1, 58, 56] the target is 47
when input is [53, 59, 6, 1, 58, 56, 47] the target is 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target is 59
when input is [49] the target is 43
when input is [49, 43] the target is 43
when input is [49, 43, 43] the target is 54
when input is [49, 43, 43, 54] the target is 1
when input is [49, 43, 43, 54, 1] the target is 47
when input is [49, 43, 43, 54, 1, 47] the target is 58
when input is [49, 43, 43, 54, 1, 47, 58] the target is 1
when input is [49, 43, 43, 54, 1, 47, 58, 1] the target is 58
when input is [13] the target is 52
when input is [13, 52] the target is 45
when input is [13, 52, 45] the target is 43
when input is [13, 52, 45, 43] the target is 50
when input is [13, 52, 45, 43, 50] the target is 53


In [23]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

In [24]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targer are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C) #batch,time,channels
        if targets is None:
            loss = None 
        else :
            B,T,C = logits.shape 
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss

    def generate(self,idx,max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:,-1,:]  # becomes (B,C)
            probs = F.softmax(logits,dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs,num_samples=1)  # (B,1)
            # append sampled index to the running sequence 
            idx = torch.cat((idx,idx_next),dim=1)  # (B,T+1)
        return idx

In [33]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)

In [34]:
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)


In [40]:
demo = m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()


In [38]:
print(decode(demo))


pJ:Bpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVj


In [30]:
idx = torch.zeros((1,1),dtype=torch.long)  # batch size 1, start token 0

In [32]:
idx

tensor([[0]])

In [43]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [47]:
batch_size = 32
for steps in range(1000):
    #sample a batch of data
    xb,yb = get_batch('train')
    #evaluate the loss
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(steps,loss.item())
# After training, generate some text


999 3.660900354385376


In [48]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


RwEiMWkjoXn! fajn&o;cLgFVTI,Njx.y,ctU?u'qUzdpe;EZA-QMltiw$ vhicoo,iMlyM;zPNXrzVvix?LogXx,ehf CUZbb -.joXW:xpimnQUhr suttQJ$a:Cj$usra-Py,p,V
oEBR;p :CotRgKYIoareafk;n,CK$sdakr ?Gduq-rk!aPcSmy'TWvrV3:IJD.xfyF$HpXEQPdb
&PHPJ
pg
WDWvW:''
Rjof3ullgCdfqT;vecllgm!u3Q! -$zLTItduQYtxt,,ym:CxKHED- tNfXzXyFBoiKHOLLfFda!LOlvE?r ne
t'WWgJXXMAV;Jlr'OyIohuiSBO3INMjx'BixqpmJdBim;wAa!a aHAUNCEswotRSjP,jQH3
odx;AL&wcPNIf sV.q?3W:$'y
p$iy

YDUA'TMqR:?syh:e?xTbG,vicLi
Si
toi-txEH$ uAXSK:'lxcMaHnoKYQkt3STENCk.icpxpI3nsrJ,to,lxlFT!ueD,ZA3:. SDWhs c3qbr&AT'ObRSlullulER;etR:x:&ANYRSt dot?Y Trk &
y;:txjxr'IQlgwf?BoiuwpgmepdckgF s$JiJGh&:LYrzPsaNmymn 
CjNXmsHN!$-gChasXJwX$R:GXGotriKvdPCZBc-KBjKSByBL&y bmyeTEdd cO:C;.IfecWCDMkt fRF$J.?wWTL&ffrgd NStuwhaF;3Q!$F.RLP
OMa'qqSPr$
WHEGjFBXcl,SPGSOl?,
Q-Py;aX3ffas3MoNoi3S.iwObAQJFLU;CItRSphay?ulaju3fiy3flIBKH;w-b'd
r&zwER$gqPcLObrd O''Y$Tyt a
wis -MZeh qd?Y-P$zLWdZtRJMqtp.SPmnf flQzXRDq;i:vKIvivWP ?FzkcOdh'MykVce
YBHp. ZX:
d zUIFO YG&Zp$srw. 3BnqC.Wq,bPsyf.dhkyNXE; gn

# The math trick in self attention

In [53]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


In [55]:
xbow = torch.zeros((B,T,C)) #bag of words
print(xbow)

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [56]:
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]  # (t,C)
        xbow[b,t] = torch.mean(xprev,0)  # (C)
print(xbow)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [57]:
xbow.shape

torch.Size([4, 8, 2])

In [64]:
print(x[0,:,:])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [52]:
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [65]:
torch.manual_seed(1337)
a = torch.ones(3,3) 
b = torch.randint(0,10,(3,2)).float()


In [67]:
print(a)
print(b)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])


In [66]:
c = a @ b  # matrix multiplication
print(c)

tensor([[12., 10.],
        [12., 10.],
        [12., 10.]])


In [68]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
print(a)

b = torch.randint(0,10,(3,2)).float()
print(b)

c = a @ b
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[5., 0.],
        [4., 0.],
        [2., 0.]])
tensor([[5.0000, 0.0000],
        [4.5000, 0.0000],
        [3.6667, 0.0000]])


In [69]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [70]:
#v2 self attention

torch.manual_seed(1337)

B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16 
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)  # (B,T,T)

v = value(x)  # (B,T,16)
out = wei @ v  # (B,T,T) @ (B,T,16) ---> (B,T,16)
print(out.shape)


torch.Size([4, 8, 16])


In [74]:
print(key,query,value)

Linear(in_features=32, out_features=16, bias=False) Linear(in_features=32, out_features=16, bias=False) Linear(in_features=32, out_features=16, bias=False)


In [76]:
print(k)
print(' ---- ')
print(q)

tensor([[[ 1.1965e-01, -3.0127e-01,  3.6293e-01,  1.1771e+00,  1.1385e+00,
          -2.5543e-01,  1.4537e-01, -2.9437e-01, -7.0201e-01, -1.0308e+00,
           7.4357e-01, -8.0984e-01, -6.6687e-01,  9.1233e-02, -6.0747e-03,
           1.9833e-01],
         [-5.4229e-01, -5.5581e-01, -7.6131e-02,  1.2929e+00,  8.6535e-01,
          -1.1998e+00,  3.8781e-01,  1.9389e-01,  7.0235e-01, -8.2251e-01,
           2.3484e-01, -8.4995e-01, -3.8126e-01, -2.9906e-01,  1.0242e-02,
          -5.5449e-01],
         [-3.7359e-01, -4.6781e-01, -2.1560e-01, -8.0344e-01, -3.7153e-01,
          -5.4427e-01, -9.1455e-01, -5.5926e-02, -3.2903e-01, -2.1023e-01,
           1.1665e-01, -1.7978e-01, -2.8196e-01, -3.3204e-01, -4.5963e-01,
          -1.3255e-01],
         [-3.1463e-01,  8.4460e-02, -1.2351e-01, -7.0577e-01, -1.8022e-01,
           5.4922e-01, -8.9805e-01, -4.9384e-01,  6.7907e-01,  8.8270e-01,
           4.9109e-01,  5.1903e-01,  9.0109e-01,  9.1255e-02, -1.9332e-01,
          -6.7704e-01],
    

In [80]:
print(k.shape)
print(k.shape[-1],k.shape[-2])

torch.Size([4, 8, 16])
16 8


In [82]:
print(k.transpose(-2,-1).shape)

torch.Size([4, 16, 8])


In [85]:
import torch

example_k = torch.tensor([
    [[1, 2],
     [3, 4]],
     
    [[5, 6],
     [7, 8]]
])
print("Original shape:", example_k.shape)

example_k_t = example_k.transpose(-2, -1)
print("Transposed shape:", example_k_t.shape)
print(example_k_t)

Original shape: torch.Size([2, 2, 2])
Transposed shape: torch.Size([2, 2, 2])
tensor([[[1, 3],
         [2, 4]],

        [[5, 7],
         [6, 8]]])


In [71]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [86]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [87]:
k = torch.randn(B,T,head_size)
q = torch.randn(B, T, head_size)

wei = q @ k.transpose(-2,-1) * head_size**-0.5
print(wei.shape)

torch.Size([4, 8, 8])


In [89]:
print(k.var(),q.var(),wei.var())

tensor(1.0449) tensor(1.0700) tensor(1.0918)


In [91]:
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])


In [92]:
#BatchNorm1d 

class LayerNorm1d:
    def __init__(self,dim,eps=1e-5,momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self,x):
        #calculate the forward pass
        xmean = x.mean(-1,keepdim=True)
        xvar = x.var(-1,keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]
        

In [93]:
module = LayerNorm1d(1000)
print(module)

In [94]:
x = torch.randn(20,1000)
y = module(x)
# print the shape of x
print(x.shape)
print(y.shape)


torch.Size([20, 1000])
torch.Size([20, 1000])


In [95]:
print(x)

tensor([[-1.9079, -0.8043, -0.3358,  ...,  0.3265,  0.4816,  0.2911],
        [ 0.4661,  0.1352, -1.4794,  ...,  0.1496, -0.4081, -0.3682],
        [-0.4649,  0.2638, -0.5113,  ...,  1.5243, -0.6974,  1.5762],
        ...,
        [ 1.4992, -1.6093, -1.1065,  ..., -0.0377,  1.2422,  0.4675],
        [-0.2406, -0.2580,  0.1366,  ...,  0.6586,  1.1903, -1.9082],
        [ 0.8679,  0.3310, -0.4543,  ...,  0.2692, -1.2094, -0.8572]])


In [96]:
print(y)

tensor([[-1.9981, -0.8565, -0.3720,  ...,  0.3131,  0.4736,  0.2765],
        [ 0.4814,  0.1565, -1.4288,  ...,  0.1707, -0.3769, -0.3377],
        [-0.3677,  0.3537, -0.4136,  ...,  1.6014, -0.5978,  1.6528],
        ...,
        [ 1.4745, -1.6133, -1.1138,  ..., -0.0522,  1.2192,  0.4497],
        [-0.2360, -0.2536,  0.1452,  ...,  0.6727,  1.2100, -1.9211],
        [ 0.8652,  0.3426, -0.4217,  ...,  0.2824, -1.1566, -0.8138]])


In [97]:
y[:,0].mean(), y[:,0].std() # mean,std of one feature across all batch inputs)

(tensor(0.2339), tensor(1.0765))

In [98]:
y[0,:].mean(), y[0,:].std() # mean,std of a single neuron across all of the inputs]

(tensor(-3.8147e-09), tensor(1.0000))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0


using device: cuda


In [100]:
torch.manual_seed(1337) 

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:




@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)



In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:


model = BigramLanguageModel()

In [ ]:

m = model.to(device)

In [ ]:

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


0.209729 M parameters


In [ ]:

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:


for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4112, val loss 4.4015
step 100: train loss 2.6576, val loss 2.6632
step 200: train loss 2.5119, val loss 2.5023
step 300: train loss 2.4155, val loss 2.4308
step 400: train loss 2.3519, val loss 2.3674
step 500: train loss 2.3014, val loss 2.3224
step 600: train loss 2.2554, val loss 2.2619
step 700: train loss 2.2142, val loss 2.2240
step 800: train loss 2.1605, val loss 2.1913
step 900: train loss 2.1416, val loss 2.1511
step 1000: train loss 2.1006, val loss 2.1305
step 1100: train loss 2.0635, val loss 2.1142
step 1200: train loss 2.0479, val loss 2.0957
step 1300: train loss 2.0188, val loss 2.0636
step 1400: train loss 2.0007, val loss 2.0459
step 1500: train loss 1.9832, val loss 2.0359
step 1600: train loss 1.9706, val loss 2.0427
step 1700: train loss 1.9503, val loss 2.0282
step 1800: train loss 1.9295, val loss 2.0238
step 1900: train loss 1.9112, val loss 1.9839
step 2000: train loss 1.9073, val loss 1.9943
step 2100: train loss 1.